In [4]:
# Clear the R environment and set working directory
rm(list = ls())
setwd("/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters")
path <- "/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters"

In [30]:
# Install and load additional packages needed
# install.packages("spatstat")
library(did)
library(rio)
library(ggplot2)
library(spatstat)

also installing the dependencies ‘deldir’, ‘polyclip’, ‘spatstat.sparse’, ‘tensor’, ‘goftest’, ‘spatstat.data’, ‘spatstat.geom’, ‘spatstat.random’, ‘spatstat.core’, ‘spatstat.linnet’, ‘spatstat.utils’





The downloaded binary packages are in
	/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T//RtmpBNPF2i/downloaded_packages


Loading required package: spatstat.data

Loading required package: spatstat.geom

spatstat.geom 2.4-0

Loading required package: spatstat.random

spatstat.random 2.2-0

Loading required package: spatstat.core

Loading required package: nlme

Loading required package: rpart

spatstat.core 2.4-4

Loading required package: spatstat.linnet

spatstat.linnet 2.3-2


spatstat 2.3-4       (nickname: ‘Watch this space’) 
For an introduction to spatstat, type ‘beginner’ 




In [38]:
# read in the data set
btw <-import(sprintf("%s/data/btw_treat.csv", path))
summary(btw)

      AGS               GEN                 year          Land          
 Min.   : 1001000   Length:141136      Min.   :1994   Length:141136     
 1st Qu.: 6433006   Class :character   1st Qu.:1998   Class :character  
 Median : 7338022   Mode  :character   Median :2005   Mode  :character  
 Mean   : 7890831                      Mean   :2006                     
 3rd Qu.: 9572121                      3rd Qu.:2013                     
 Max.   :16077055                      Max.   :2017                     
                                                                        
    state_id          Kreis          Wahlkreis     post_1998     
 Min.   : 1.000   Min.   : 101.0   Min.   :  1   Min.   :0.0000  
 1st Qu.: 6.000   1st Qu.: 631.0   1st Qu.: 63   1st Qu.:0.0000  
 Median : 7.000   Median : 743.0   Median :196   Median :1.0000  
 Mean   : 7.616   Mean   : 814.3   Mean   :161   Mean   :0.7326  
 3rd Qu.: 9.000   3rd Qu.: 977.0   3rd Qu.:226   3rd Qu.:1.0000  
 Max.   :16.000   Ma

In [39]:
state_dummies <- dummify(btw$Land)

In [40]:
btw <- cbind(btw, state_dummies)

In [43]:
colnames(btw)

[1] "AGS"           "GEN"           "year"          "Land"         
 [5] "state_id"      "Kreis"         "Wahlkreis"     "post_1998"    
 [9] "post_2002"     "post_2005"     "post_2009"     "post_2013"    
[13] "first_vote"    "second_vote"   "Wähler"        "Gültig"       
[17] "Union"         "SPD"           "FDP"           "Linke"        
[21] "Grüne"         "Andere"        "treatment_05"  "treatment_09" 
[25] "treatment_13"  "treatment_30"  "treatment_60"  "treatment_100"
[29] "BB"            "BE"            "BW"            "BY"           
[33] "HB"            "HE"            "HH"            "MV"           
[37] "NI"            "NW"            "RP"            "SH"           
[41] "SL"            "SN"            "ST"            "TH"

In [42]:
# Filter for first and second vote
erst <- subset(btw, first_vote == 1)
zweit <- subset(btw, second_vote == 1)

In [46]:
parties <- c('Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere')
treatments <-  c('treatment_30', 'treatment_60', 'treatment_100')

for(party in parties){
  for(treatment in treatments){
    out <- att_gt(yname = party,
                  gname = treatment,
                  idname = 'AGS',
                  panel = TRUE,
                  tname = 'year',
                  xformla = ~ 1, # C(Land),
                  data = erst,
                  est_method = 'dr',
                  anticipation = 0,
                  control_group = 'nevertreated',
                  clustervars = c('AGS', 'state_id'),
                  #bstrap = TRUE,
                  #cband = TRUE,
                  allow_unbalanced_panel = TRUE,
    )
    # summary(out)
    ggdid(out, ylim = c(floor(min(out$att - out$se * 2.345 - 1)), ceiling(max(out$att + out$se * 2.345 + 1))))
    ggsave(sprintf('%s/figures/R/Group/%s_%s.png', path, party, treatment))
    es <- aggte(out, type = 'dynamic')
    # summary(es)
    ggdid(es, ylim = c(floor(min(es$att.egt - es$se.egt * 2.345 - 1)), ceiling(max(es$att.egt + es$se.egt * 2.345 + 1))))
    ggsave(sprintf('%s/figures/R/ES/ES_%s_%s.png', path, party, treatment))
  }
}

ERROR: Error in DRDID::drdid_rc(y = Y, post = post, D = G, covariates = covariates, : Propensity score model coefficients have NA components. 
 Multicollinearity (or lack of variation) of covariates is a likely reason.
